In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-20-questions/llm_20_questions/llm_20_questions.json
/kaggle/input/llm-20-questions/llm_20_questions/llm_20_questions.py
/kaggle/input/llm-20-questions/llm_20_questions/keywords.py
/kaggle/input/llm-20-questions/llm_20_questions/llm_20_questions.js


# 1. 환경 설정 및 데이터 불러오기


In [2]:
%%bash
mkdir ./submission

In [ ]:
# main.py for submission
%%writefile -a ./submission/main.py

import os
import sys

# import Tokenizer and Model
from transformers import AutoTokenizer, AutoModelForCasualLM
import torch
from gemma.config import get_config_for_9b
from gemma.model import GemmaForCasualLM


# Set Agent Path
KAGGLE_AGENT_PATH = './kaggle_simulations/agent/'

if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, '/kaggle/working/submission/lib')
    
if os.path.exsits(KAGGLE_AGENT_PATH):
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, 'gemma/pytorch/9b')
else:
    WEIGHTS_PATH = '/kaggle/input/gemma/pytorch/9b'


# 파일 살펴보기

데이터셋이 업데이트되었다.
kaggle-environments/kaggle_environments/envs/llm_20_questions
/keywords.py
(https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/llm_20_questions/keywords.py)
- 확인해보면 country, city, landmark 가 전부 place로 바뀌었고,
- things가 대거 추가되었다고 한다.

In [ ]:
import json
from pprint import pprint as pp

file_path = '/kaggle/input/llm-20-questions/llm_20_questions/llm_20_questions.json'

with open(file_path) as json_file:
    json_data = json.load(json_file)

In [ ]:
# 키워드 파일 경로 추가
import sys

keywords_path = "/kaggle/input/llm-20-questions/llm_20_questions/"
sys.path.insert(0, keywords_path)

# 키워드 파일 임포트 시도
try:
    import keywords
    print("keywords 모듈 임포트 성공!")
except ModuleNotFoundError:
    print("keywords 모듈을 찾을 수 없습니다. 경로를 확인하세요.")


In [ ]:
# Keywords가 어떻게 생긴 파일인지 조금 더 살펴보자 (EDA)
file_path = "/kaggle/input/llm-20-questions/llm_20_questions/keywords.py"

import json

# json file의 형태로 되어있어서, 이것만 골라내려 함.
with open(file_path, 'r') as f:
    file_contents = f.read()
    
    # JSON 데이터만 추출
    start_idx = file_contents.find('KEYWORDS_JSON = """') + len('KEYWORDS_JSON = """')
    end_idx = file_contents.find('"""', start_idx)
    json_data_str = file_contents[start_idx:end_idx].strip()
    
    # JSON parsing
    json_data = json.loads(json_data_str)
                        

In [ ]:
pp(json_data[0])

In [ ]:
# category 별 분포.
# 첫 질문이 is it a city? 로만 시작해도 절반은 거르고 갈 수 있음
categories = [(item['category'], len(item['words'])) for item in json_data]
print(categories)

In [ ]:
json_data[1]['words']